In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install tensorflow

In [ ]:
!pip install ipython

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
!pip install pyyaml h5py 

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import pathlib
import shutil
import tempfile
from IPython import display

In [ ]:
import keras_tuner as kt

In [ ]:
print(os.getcwd())
# Change current directoru to dataset folder structure directory
os.chdir('D:\Load_Data_Category_32')
print(os.getcwd())
# class names
class_names=os.listdir() 
# no of classes
num_class= len(class_names)
print(class_names)
print(num_class)

In [ ]:
img_height=32
img_width=32
batch_size=32
epochs=10

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.getcwd(),
    labels='inferred',
    label_mode='int',
    class_names= class_names,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_width,img_width),
    shuffle=True,
    seed=10,
    validation_split=0.1,
    subset="training",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
plt.figure(figsize=(5,5))
for image, label in train_ds.take(1):
    for x in range(9):
        ax=plt.subplot(3, 3, x+1)
        plt.imshow(image[x].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[label[x].numpy()])
        plt.axis("off")
plt.show()

In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    os.getcwd(),
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=10,
    validation_split=0.1,
    subset="validation",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
plt.figure(figsize=(5,5))
for image, label in validation_ds.take(1):
    for x in range(9):
        ax=plt.subplot(3, 3, x+1)
        plt.imshow(image[x].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[label[x].numpy()])
        plt.axis("off")
plt.show()

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip(input_shape=(img_height,img_width,1), mode='horizontal'),
            tf.keras.layers.RandomRotation(0.1),
])

In [ ]:
def build_model(hp):
    
    units_1 = hp.Int('units_1',min_value=32, max_value=512, step=32)
    units_2 = hp.Int('units_2',min_value=32, max_value=512, step=32)
    units_3 = hp.Int('units_3',min_value=32, max_value=512, step=32)
    units_4 = hp.Int('units_4',min_value=32, max_value=512, step=32)
    
    units_5 = hp.Int('units_5',min_value=32, max_value=512, step=32)
    units_6 = hp.Int('units_6',min_value=32, max_value=512, step=32)
    
    dropout_1 = hp.Boolean('dropout_1')
    dropout_2 = hp.Boolean('dropout_2')
    dropout_3 = hp.Boolean('dropout_3')
    
    lr = hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='log')
    
    model = tf.keras.Sequential()
    model.add(data_augmentation)
    model.add(tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)))
    model.add(tf.keras.layers.Conv2D(units_1, 3, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(units_2, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    if dropout_1:
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Conv2D(units_3, 3, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(units_4, 3, activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D())
    if dropout_2:
        model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=units_5, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    if dropout_3:
        model.add(tf.keras.layers.Dropout(0.2)) 
    model.add(tf.keras.layers.Dense(units=units_6, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
    model.add(tf.keras.layers.Dense(num_class, activation='softmax'))
    
    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
    )
    
    return model

In [ ]:
build_model(kt.HyperParameters())

In [ ]:
tuner=kt.RandomSearch(
                  hypermodel=build_model,
                  objective="val_accuracy",
                  max_trials=5,
                  executions_per_trial=1,
                  overwrite=True,
                  directory='D:\Robotics\Tuner_Results',
                  project_name="Test_01",
)

In [ ]:
tuner.search_space_summary()

In [ ]:
stop_early=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(train_ds, epochs=5, validation_data = validation_ds, callbacks=[stop_early])


In [ ]:
models=tuner.get_best_models(num_models=2)
best_model=models[0]
best_model.build(input_shape=(None, img_height, img_width))
best_model.summary()

In [ ]:
tuner.results_summary()